<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">
           <a href="https://bokeh.org/" target="_blank">
           <img
               src="assets/bokeh-transparent.png"
               style="width:50px"
           >
           </a>
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><a href="TOC.ipynb" target="_blank">Table of contents</a><br><h2>11 Widgets and interactivity</h2></div>

In [ ]:
# load tutorial data
from tutorial_data import data

In [ ]:
# activate notebook output
from bokeh.io import output_notebook

output_notebook()

In this chapter, you learn how to **make your Bokeh plots interactive**. This includes
making parts of your plot clickable and adding widgets to your plots. This way, you
can create user interfaces that allow your users to interact with your data.

### Simple interactions: URLs and interactive legends

Bokeh allows you to define complex interactions with custom JavaScript callback code.
However, there are also several **simple interaction types** that you can use without
writing any JavaScript.

In this tutorial, you will learn how to use the following interaction types:
*  **[OpenURL](11_widgets_interactivity.ipynb#OpenURL)**: You can define URLs that are
  opened when a user clicks on a glyph.
* **[Interactive legends](11_widgets_interactivity.ipynb#Muting-and-hiding-glyphs)**:
  You can make your legend clickable. This will mute or hide the corresponding glyphs.

#### OpenURL

The `OpenURL` class allows you to **define URLs that are opened when a user clicks on a
glyph**.

Let's create a DataFrame with population data for the world's five largest countries.
This DataFrame also includes a Wikipedia URL for each country:

In [ ]:
import pandas as pd

population_df = pd.DataFrame(
    {
        "country": ["China", "India", "USA", "Indonesia", "Pakistan"],
        "population": [1412600000, 1375586000, 333340028, 275773800, 235825000],
        "wikipedia_url": ["https://en.wikipedia.org/wiki/China", "https://en.wikipedia.org/wiki/India", "https://en.wikipedia.org/wiki/United_States", "https://en.wikipedia.org/wiki/Indonesia", "https://en.wikipedia.org/wiki/Pakistan"],
    }
)
population_df

Create a bar chart with the population data. Each bar is clickable and opens
the corresponding Wikipedia page.

This code uses the `OpenURL` class to define the URL that is opened when a user
clicks on a bar. 

The `OpenURL` class takes a `url` parameter that defines the URL
template. The template can contain placeholders that are replaced with the values
of the data source columns. In this example, the `@wikipedia_url` placeholder is
replaced with the value of the `wikipedia_url` column in the DataFrame.

To make the bars clickable, you also need to have the tap tool enabled. This is
done by adding `"tap"` to the `tools` parameter of the `figure()` function.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import NumeralTickFormatter, OpenURL, TapTool, ColumnDataSource

p = figure(
    x_range=population_df["country"],
    height=350,
    title="Most populous countries",
    tools="tap",  # enable the tap tool to make the bars clickable
)
p.vbar(x="country", top="population", width=0.9, source=population_df)
p.yaxis.formatter = NumeralTickFormatter(format="0,0")

# Configure the OpenURL as the tap tool's callback
url = "@wikipedia_url"  # use the URLs stored in the wikipedia_url column
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)  # define the OpenURL object as the tap tool's callback

show(p)

#### OpenURL example

The "Top 10 carriers by passengers" plot in the demo dashboard uses OpenURL. You can
click on a bar to open the IATA's page for the corresponding airline.

The DataFrame for this board contains a `unique_carrier` column. This column contains
the IATA code for each airline:

In [ ]:
data.get_biggest_airlines_by_passengers().head()

The url for the OpenURL object uses a field to add the code to the IATA URL:

```python
url = "https://www.iata.org/en/publications/directories/code-search/?airline.search=@unique_carrier"
```

For example: for "Southwest Airlines Co.", the IATA code in the `unique_carrier` column
is "WN". This means that
https://www.iata.org/en/publications/directories/code-search/?airline.search=WN will be
the URL.

The code cell below shows the code for the complete plot, including the OpenURL object:

In [ ]:
from bokeh.models import NumeralTickFormatter, TapTool

# load data for ColumnDataSource from demo data set
source = ColumnDataSource(data.get_biggest_airlines_by_passengers())

# Define tooltips as a list of tuples
TOOLTIPS = [
    ("Position", "@position"),
    ("Carrier", "@unique_carrier_name"),
    ("Passengers", "@passengers{(0,0)}"),
]

# set up a figure with the tooltips to use the TOOLTIPS list
largest_carriers_plot = figure(
    x_range=source.data["unique_carrier_name"][:10],
    title="Top 10 carriers by passengers (domestic routes)",
    height=300,
    tooltips=TOOLTIPS,
    tools="tap, hover",
    toolbar_location=None,
)

# create a vbar renderer with the data from the ColumnDataSource
carriers_vbar = largest_carriers_plot.vbar(
    x="unique_carrier_name",
    top="passengers",
    nonselection_alpha=0.8,
    source=source,
    legend_label="Passengers",
    width=0.6,
)

# customize the appearance of the grid and x-axis
largest_carriers_plot.xgrid.grid_line_color = None
largest_carriers_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
largest_carriers_plot.xaxis.major_label_orientation = 0.8  # rotate labels by roughly pi/4

# Add TapTool to look up airline IATA code on click
url = "https://www.iata.org/en/publications/directories/code-search/?airline.search=@unique_carrier"
taptool = largest_carriers_plot.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(largest_carriers_plot)

For more information about ``OpenURL``, see [OpenURL](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html#openurl) in the user guide.

#### Muting and hiding glyphs

Bokeh also includes a quick way to make a **legend clickable**. This allows you to mute
or hide the corresponding glyphs.

To make a legend clickable, use the legend's `click_policy` parameter. This parameter
can have the following values:

* **`mute`**: This mutes the corresponding glyphs. This means that the glyphs are
   still visible, but they are grayed out.
* **`hide`**: This hides the corresponding glyphs. This means that the glyphs are
    not visible anymore.
* **`none`**: The legend is not clickable. This is the default value.


This example uses the [sampledata that comes with Bokeh](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#bokeh-sampledata).
You can download it with the following code:

In [ ]:
import bokeh.sampledata

bokeh.sampledata.download()

In the code cell below, change the `click_policy` parameter from `"mute"` to `"hide"`.
Rerun the cell and try the different behaviors.

In [ ]:
import pandas as pd

from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show
from bokeh.sampledata.stocks import AAPL, GOOG, IBM, MSFT

p = figure(width=800, height=250, x_axis_type="datetime")
p.title.text = "Click on legend entries to hide the corresponding lines"

for stocks, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
    df = pd.DataFrame(stocks)
    df["date"] = pd.to_datetime(df["date"])
    p.line(df["date"], df["close"], line_width=2, color=color, alpha=0.8, legend_label=name)

p.legend.location = "top_left"
p.legend.click_policy = "hide"  # 🔁 change to "mute" to mute the lines instead of hiding them

show(p)

#### Interactive legends example

The plot "Distance flown vs. number of passengers, freight, and mail" is a massive
scatter plot with almost 225.000 points. These points use different glyphs for the
three different types of data:

* circle glyphs for passengers
* square glyphs for freight
* triangle glyphs for mail

The plot's legend uses the `click_policy` parameter set to `"hide"`. This means that
a user can hide any of the three glyph types by clicking on the corresponding legend.
This makes the plot a lot more readable.

The following code cell contains the full code for this plot. Due to the large number
of points, the plot takes a few seconds to render:

In [ ]:
from bokeh.palettes import Category10

# define a list of markers to use for the scatter plot
MARKERS = ["circle", "square", "triangle"]

source = ColumnDataSource(data.get_distance_df())

# set up the tooltips
TOOLTIPS = [
    ("Distance", "@distance{(0,0)} miles"),
    ("Route", "@origin, @dest"),
    ("Amount", "$y{(0,0)}"),
]

# set up the figure
distance_plot = figure(
    title="Distance flown vs. number of passengers, freight, and mail",
    height=300,
    sizing_mode="stretch_width",  # use the full width of the parent element
    tooltips=TOOLTIPS,
    output_backend="webgl",  # use webgl to speed up rendering (https://docs.bokeh.org/en/latest/docs/user_guide/output/webgl.html)
    tools="pan,box_zoom,reset,save",
    active_drag="box_zoom",  # enable box zoom by default
)

# loop through the three metrics ("passengers", "freight", "mail") and plot them
i = 0
for metric in data.metrics:
    distance_plot.scatter(
        "distance",
        metric,
        source=source,
        legend_label=metric.capitalize(),
        color=Category10[3][i],  # assign a different color to each metric
        marker=MARKERS[i],  # assign a different marker to each metric
        alpha=0.5,
    )
    i += 1

# customize plot appearance
distance_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
distance_plot.xaxis.axis_label = "Distance (miles)"
distance_plot.legend.click_policy = "hide"  # set the legend click policy to hide

show(distance_plot)

### Custom interactions: Widgets and JavaScript callbacks

The interactive examples you saw so far (OpenURL and interactive legends) are
pre-defined interactions. However, Bokeh also allows you to define custom interactions.

Bokeh interactions are a complex topic. This tutorial will only cover some basics to get
you started. A lot more information is available in the
[Interaction](https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html) section
of the user guide.

#### Widgets

Before diving into interactions, let's first look at the **interactive elements you can
use to build user interfaces**.

So far, you have built different plots. However, Bokeh also includes a large library of
user interface (UI) elements. These UI elements are called **widgets**.

A lot of those widgets are designed to receive user input. For example: a button, a
text field, or a slider.

All widgets are importable from the `bokeh.models` module. The following code is an
example of a CheckboxButtonGroup:

In [ ]:
from bokeh.models import CheckboxButtonGroup

LABELS = ["Option 1", "Option 2", "Option 3"]

checkbox_button_group = CheckboxButtonGroup(labels=LABELS, active=[0, 1])

show(checkbox_button_group)

Each widget has its own parameters to customize its behavior. For example, the following
parameters are specific for the functionality of a Slider widget:

* `start`: The minimum value of the slider.
* `end`: The maximum value of the slider.
* `step`: The step size by which a user can increase or decrease the value.
* `value`: The initial value of the slider.

Use the following cell to try different values for these parameters:

In [ ]:
from bokeh.models import Slider

# 🔁 change the start, end, value, and step parameters and rerun this cell to see how they affect the slider
slider = Slider(start=0, end=10, value=1, step=0.1, title="Slider Example")

show(slider)

In addition to widgets that receive user input, Bokeh also includes widgets that
display information. For example:

* The `Paragraph` widget displays text.
* The `Div` widget displays HTML content.

The following code cell contains an example of a `Div` widget:

In [ ]:
from bokeh.models import Div

div = Div(
    text=r"""
<div style="background-color: darkolivegreen; padding:10px;">
    <h2>Bokeh's Div widget</h2>
    <p> Bokeh's Div widget uses <a href="https://en.wikipedia.org/wiki/HTML">HTML</a> to display text and other information.</p target="_blank">
    <p> It is one of the elements that support <a href="https://docs.bokeh.org/en/latest/docs/user_guide/styling/mathtext.html">LaTeX and MathML math expressions</a>. For example:</p target="_blank">
    <p>$$\sin^2(x) + \cos^2(x) = 1$$</p>
</div>
""",
)

show(div)

Bokeh handles widgets in a similar way to plots. You already saw that you use the same
`show()` function to display widgets. Widgets become much more useful when you use them
in [layouts](10_layouts.ipynb).
The following example combines an explanation in a Div widget with two
plots:

In [ ]:
import numpy as np
from bokeh.layouts import layout

# create a div widget with explanatory text
div = Div(
    text="""
<h1>Sine and cosine waves</h1>
<p>The main difference between the sine and cosine functions is that the sine function
has a phase shift of 90 degrees, while the cosine function has a phase shift of 0 degrees.
This means that the sine function starts at a maximum value and oscillates between
positive and negative values, while the cosine function starts at a minimum value and
oscillates between 0 and a positive maximum value.</p>
"""
)

# create a plot demonstrating a sine wave
p1 = figure(height=300)
x = np.linspace(0, 10, 100)
y = np.sin(x)
p1.line(x, y, line_width=7, line_color="orange")
p1.xaxis.fixed_location = 0

# create a plot demonstrating a cosine wave
p2 = figure(height=300)
y = np.cos(x)
p2.line(x, y, line_width=7, line_color="darkgreen")
p2.xaxis.fixed_location = 0

# define the layout
layout = layout(
    [
        [div],
        [p1, p2],
    ],
    sizing_mode="stretch_width",
)

show(layout)

#### Pre-defined callbacks

Bokeh uses callbacks to **react to user interactions**. A callback is a piece of code
that is **executed when a user interacts with a widget or a plot**.

Bokeh includes some pre-defined callbacks that you can use without writing any
JavaScript. For example:

* Use the `js_link()` method to link the properties of two objects.
* Use a `SetValue` object to set the value of a property in one object from
  another object.

##### Linking properties with `js_link()`

The `js_link()` method allows you to **control a property of one object with input from
another object**.

For example, you can use `js_link()` to link the value of a slider to the ``size``
property of a circle glyph. This means that the user can change the diameter of the
circle by moving the slider:

In [ ]:
from bokeh.layouts import column
from bokeh.models import Slider

plot = figure(width=400, height=200)
circles = plot.scatter([1, 2, 3, 4, 5], [3, 2, 5, 6, 4], size=25, alpha=0.5)

slider = Slider(start=1, end=100, step=1, value=25, title="Circle diameter", sizing_mode="stretch_width")
slider.js_link("value", circles.glyph, "size")  # add a js_link between the slider and the circle size

show(column(slider, plot))

Let's walk through the details of how this works:

For this example, it is important to **assign a variable name to the circle
glyphs**.
In most previous examples, you just called a method to add a glyph to a plot.
For example: `p.scatter(x=x, y=y)`.
In order for `js_link()` to work, you need to be able to reference the glyph.
For this reason, you need to assign a variable name when you call the glyph method.
For example: `circles = p.scatter(x=x, y=y)`.
This allows you to **reference this glyph renderer later in your code**. The glyphs
themselves are accessible through the **`glyph` attribute of a glyph renderer**.
For example: `circles.glyph`.

Now that you can access the circle glyph and its properties, let's look at the
line where you call the `js_link()` method:

```python
slider.js_link("value", circles.glyph, "size")
```

First of all, the `js_link()` method is called on the object you want to read the data
from. This is the **source object**.
In this case, you want to read data from the ``slider`` object.
This is why the `js_link()` method is called on ``slider``.

The `js_link()` method takes three parameters:
1. The name of the **property to read from the source object**. In this case, you want to
   read the current value of the slider. This value is stored in the slider's ``value``
   property.
2. The object you want to write data to. This is the **target object**. In this case,
   you are writing values to the ``circle`` glyph object. The properties of a glyph
   renderer's glyphs are accessible through the ``glyph`` attribute. This is why you use
   ``circles.glyph``.
3. The specific **property of the target object you want to write to**. In this case,
   you want to change the ``size`` property of the circle glyphs. This is why you use
   ``"size"``.

For more information on the `js_link()` method, see
[Linked properties](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html#linked-properties)
in the user guide.

##### Setting values with `SetValue`

The `js_link()` method continuously updates the values of the target object every time
the linked value of the source object changes.

The `SetValue` object works in a similar way. The main difference is that it is **event-
driven**.

Many Bokeh objects emit events. For example, when a user clicks on a button. You can
define actions that happen when a specific event is emitted.

The `SetValue` object is one of those actions that you can link to an event.

In the following example, you use the `SetValue` object to change the value of the
Button's ``label`` property from "Foo" to "Bar". This event is triggered when a user
clicks on the button:

In [ ]:
from bokeh.io import show
from bokeh.models import Button, SetValue

button = Button(label="Foo", button_type="primary")
action = SetValue(button, "label", "Bar")
button.js_on_event("button_click", action)

show(button)

Let's look at the details of how SetValue works in this example:

First, you **create a Button object** and assign it to the variable ``button``.

Second, you **define the action you want to perform** when an event happens.
In this case, you use a ``SetValue`` object to define the action.
The SetValue object takes three parameters:

1. The **object you want to change a property of**. In this case, you want to change the
    value of the ``button`` object.
2. The **property of the object you want to change**. In this case, you want to change
    the value of the ``label`` property of the button.
3. The **value you want to update the property to**. In this case, you want to change
    the value of the ``label`` property to ``"Bar"``.

Next, you **call the ``js_on_event()`` method on this button object**.
This method enables event-driven actions on this button object.
The ``js_on_event()`` method takes two parameters:
1. The name of the event you want to listen to. In this case, you want to listen to
   the ``"button_click"`` event. This event is emitted when a user clicks on the button.
2. The action you want to perform when the event is emitted. In this case, you use a
   `SetValue` object to define the action.

For a list of all events available in Bokeh, see
[bokeh.events](https://docs.bokeh.org/en/latest/docs/reference/events.html) in the
reference guide.

#### Custom JavaScript callbacks

The `SetValue` object you just saw is a pre-defined callback. It has a clear structure
and accomplishes a specific task: change a specific value of a specific object.

In some cases, you might want to **define a more flexible callback**.
This is where the `CustomJS` object comes in.
You can use it to define callbacks, just like you did with `SetValue`.
But with `CustomJS`, you can use your own JavaScript code to define a completely custom
callback.

The following example uses a `CustomJS` object to display a JavaScript alert when a user
clicks on a button:

In [ ]:
from bokeh.models import Button, CustomJS

button = Button(label="Click this button")  # define a button
action = CustomJS(code="alert('You clicked the button!')")  # define an action with custom JavaScript code
button.js_on_event("button_click", action)  # add the action to the button's "button_click" event

show(button)

Bokeh's `CustomJS` callbacks become even more powerful when you use its `args`
parameter. This parameter allows you to pass additional arguments to your JavaScript
code. The `args` parameter takes a dictionary as an argument:

* The keys of this dictionary are the names of the arguments you want to pass to your
  JavaScript code.
* The values of this dictionary are the Bokeh objects you want to pass to your
JavaScript code.

You can pass any Bokeh object to your JavaScript code, including plots or a
ColumnDataSource. This way, you can interact with Bokeh objects from your JavaScript code.

You have already learned about the ``js_on_event()`` method. This method allows you
to perform actions when a specific event happens. Bokeh has a second method to trigger
actions: the ``js_on_change()`` method. It works very similarly to ``js_on_event()``.
However, it allows you to trigger actions when a specific property of a Bokeh object
changes.

In the following example, you use the ``js_on_change()`` method to trigger custom
JavaScript code when the ``value`` property of a slider changes. This custom
JavaScript code is defined in the ``CustomJS`` object. It receives the plot's
ColumnDataSource as an argument. This way, the JavaScript code can access and
manipulate the data in the plot:

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider

x = [x * 0.005 for x in range(0, 200)]
y = x

# create a ColumnDataSource called "source"
source = ColumnDataSource(data=dict(x=x, y=y))

# set up the figure
plot = figure(width=400, height=200, x_range=(0, 1), y_range=(0, 1))

# set up a line renderer using the "source" ColumnDataSource
plot.line("x", "y", source=source, line_width=3, line_alpha=0.6)

# Set up a CustomJS callback
callback = CustomJS(
    args=dict(source=source),  # pass the ColumnDataSource as an argument to the callback so that it can be modified by the JavaScript code
    code="""
const f = this.value  // in JavaScript, the model that triggered the callback is accessible as the "this" variable
const x = source.data.x  // the ColumnDataSource is accessible as the source variable that was passed as the args argument to the CustomJS callback
const y = Array.from(x, (x) => Math.pow(x, f))
source.data = { x, y }  // update the ColumnDataSource with the new data
    """,
)

slider = Slider(start=0.1, end=4, value=1, step=0.1, title="power", sizing_mode="stretch_width")
slider.js_on_change("value", callback)

layout = column(slider, plot)

show(layout)

For more information on the `CustomJS` object, see
[CustomJS callbacks](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html#customjs-callbacks)
in the user guide.

More examples are available in the Bokeh gallery, such as the
[slider](https://docs.bokeh.org/en/latest/docs/examples/interaction/js_callbacks/slider.html),
[color_sliders](https://docs.bokeh.org/en/latest/docs/examples/interaction/js_callbacks/color_sliders.html)
, or the [customjs_lasso_mean](https://docs.bokeh.org/en/latest/docs/examples/interaction/js_callbacks/customjs_lasso_mean.html)
examples.

### CustomJS example

The "Top carriers by passengers" plot in the airline demo dashboard contains a CustomJS
callback. It also uses the OpenURL callback.

The plot is combined with a slider widget. This slider allows the user to
change how many carriers to consider. The CustomJS callback is triggered whenever
the slider's value changes. It updates the plot's ``x_range`` to display additional or
fewer carriers.

The following cell contains the full code for this plot. This includes the CustomJS
callback and the OpenURL callback:

In [ ]:
# define the initial number of carriers to display
initial_carriers = 10

# read date from the demo data set
source = ColumnDataSource(data.get_biggest_airlines_by_passengers())

# set up tooltips
TOOLTIPS = [
    ("Position", "@position"),
    ("Carrier", "@unique_carrier_name"),
    ("Passengers", "@passengers{(0,0)}"),
]

# set up the figure
largest_carriers_plot = figure(
    x_range=source.data["unique_carrier_name"][:initial_carriers],  # initially, display the top 10 carriers as the plot's x_range
    title=f"Top {initial_carriers} carriers by passengers (domestic routes)",  # initially, display 10 as the number of carriers in the title
    height=300,
    sizing_mode="stretch_width",
    tooltips=TOOLTIPS,
    tools="tap, hover",
    toolbar_location=None,
)

# add a vbar renderer
carriers_vbar = largest_carriers_plot.vbar(
    x="unique_carrier_name",
    top="passengers",
    nonselection_alpha=0.8,
    source=source,
    legend_label="Passengers",
    width=0.6,
)

largest_carriers_plot.xgrid.grid_line_color = None  # remove grid lines
largest_carriers_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")  # format y-axis ticks
largest_carriers_plot.xaxis.major_label_orientation = 0.8  # rotate labels by roughly pi/4

# Add TapTool to look up airline IATA code
url = "https://www.iata.org/en/publications/directories/code-search/?airline.search=@unique_carrier"
taptool = largest_carriers_plot.select(type=TapTool)
taptool.callback = OpenURL(url=url)

# Set up Slider widget
number_slider = Slider(
    start=1,
    end=25,
    value=len(largest_carriers_plot.x_range.factors),
    title="Number of airlines to consider",
)

# Set up CustomJS callback
custom_js = CustomJS(
    args={  # the args parameter is a dictionary of the variables that will be accessible in the JavaScript code
        "largest_carriers_plot": largest_carriers_plot,  # the first variable will be called "largest_carriers_plot" and links to the largest_carriers_plot Python object
        "carriers": source.data["unique_carrier_name"],  # the second variable will be called "carriers" and links to list of carrier names in the source ColumnDataSource
    },
    code="""
    largest_carriers_plot.title.text = "Top " + this.value + " carriers by passenger (domestic routes)"  // update the plot title using the slider's value (this.value)
    largest_carriers_plot.x_range.factors = carriers.slice(0,this.value)  // update the plot's x_range using data from the list of carrier names and the slider's value (this.value)
    """,
)

# Add callback to slider widget
number_slider.js_on_change("value", custom_js)

# assemble the layout
largest_carriers_layout = column([number_slider, largest_carriers_plot], sizing_mode="stretch_width")

show(largest_carriers_layout)

# Next section

<a href="12_demo_dashboard.ipynb" target="_blank">
    <img src="assets/arrow.svg" alt="Next section" width="100" align="right">
</a>

In the [next chapter](12_demo_dashboard.ipynb), you will combine all the concepts you
have learned in this tutorial to build a complete dashboard.